---
title: "Three.js - 3D Animations in the Browser"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["Three.js", "Visualization", "JavaScript"]
format:
  html:
    css: ./css/three-js-demo.css
    code-fold: false
toc-depth: 4
jupyter: python3
highlight-style: github
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="importmap">
      {
          "imports": {
              "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
              "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js",
              "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js"
          }
      }
    </script>
    <script type="module" src="./javascript/three-js-demo.js"></script>
    <script type="module" src="./javascript/three-js-grid-demo.js"></script>
    <script type="module" src="./javascript/three-js-custom-object-with-material-demo.js"></script>
    <script type="module" src="./javascript/three-js-many-ducks-demo.js"></script>
---

# Introduction

In this post we use [Three.js](https://threejs.org/) external scripts to render 3D scenes inside this Jupyter notebook. 

This post will be very similar, in terms of its stack, to the [D3.js interactive US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb).

In *that* post, we learned that Canvas and SVG are two ways in which we can display complex graphics inside a web browser. We already explored SVG graphics in [D3.js interactive US map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb). It's worth noting that a lot of the same functionality could've been replicated using the HTML Canvas element instead of SVG (which we ultimately chose for its superior interactive capabilities) -- the article mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). 

We ended up using [linkedom](https://github.com/WebReflection/linkedom#readme) to add a DOM API on top of the Deno environment. 

In fact, what I realized later on is that we could've simply used the Markdown inside our Jupyter notebook to create the `<svg>` element without the need to introduce a third-party DOM API on top of a browser-less JavaScript environment (Deno). Then, we could have simply fetched the `TopoJSON` data and constructed the map inside our external scripts with the rest of the complex D3 animations that had to be added as external scripts. 
These scripts are run by Quarto only *after* the page has been rendered to the browser, so we can easily reference the DOM elements we create inside our notes by `class` or `id`. Crucially, the external scripts are meant to run *inside the browser* (as opposed to being pre-computed in the browser-less Deno environment). Inside the browser they're able to leverage the existing DOM API provided by the browser's engine (e.g. `document.getElementById`). Hence, this way, we eliminate the need for Deno as well as Linkedom.

We will use the [IPython.display](https://ipython.readthedocs.io/en/8.26.0/api/generated/IPython.display.html) module to create HTML elements inside our notes rather than just using Markdown directly.

# Three.js, WebGL, and Canvas API

[Three.js](https://threejs.org/) is a library for drawing 3D graphics in the browser using JavaScript and [WebGL](https://get.webgl.org/) (see [wiki](https://www.khronos.org/webgl/wiki/Main_Page)). 

WebGL runs in an HTML Canvas (i.e. `<canvas>`). 

From the WebGL wiki: 

> WebGL is a DOM API, which means that it can be used from any DOM-compatible language: e.g. JavaScript

Three.js is a library that provides conveniences in JavaScript that abstract much of this [WebGL](https://developer.mozilla.org/en-US/docs/Web/API/WebGL_API) DOM API calls behind friendly JavaScript. 

Note that WebGL and Canvas API are two ways in which the browser draws graphics, but they both require and HTML Canvas to draw on. Three.js offers a WebGL renderer as well as a Canvas renderer. Canvas API is usually a fallback option for when WebGL, the more powerful of the two APIs, isn't available. 

Note, also, that Three.js isn't suitable for modelling purposes, for that we can use [Blender](https://www.blender.org/) or a number of other closed-source 3D modelling applications. We can even download or purchase third-party models from vendors.

Canvas and SVG elements are two ways in which we can display complex graphics inside a web browser. We already explored SVG graphics in the [D3.js interactive US Map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb) post. It's worth noting that a lot of the same functionality could've been achieved by using the HTML Canvas element with D3 instead. The article mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). In this post we use Three.js (not D3) to draw inside a Canvas element using, not the Canvas API, but rather WebGL.

# 3D Scenes - A Primer

In *any* 3D scene, be it in the web browser, inside a game engine, in a movie, in a 3D modelling application, etc. there are a bunch of **geometries**, or **shapes** that are packaged with **materials** as **meshes**. The materials can describe simple properties like reflectivity, opacity, refraction index, etc. or they can be  **textures** which are just simple [raster](https://en.wikipedia.org/wiki/Raster_graphics) images. We can also apply **shaders** to our objects, which are complex mappings of pixels (or vertices) that make up an interesting animation.
 
A scene will also have one or more **light sources**, and a **camera** to *serve* the scene in some perspective.

## Meshes, Objects, Geometries, and Materials

An `object`, for the foreseeable future, means an object file of the [OBJ Wavefront format](https://en.wikipedia.org/wiki/Wavefront_.obj_file). These are files that consist of `object.children[n].geometry` and `object.children[n].material` fields. However, the material fields inside an OBJ are just *references* (via `usemtl` statements) to the true materials which come separately in `.mtl` files (typically from the same place as the `.obj` file).

There's usually no need to break the object down into individual child geometries and their corresponding objects. But, it's certainly possible. One use case would be if we want to apply a different transformation to each component. Usually, however, we load the object as a whole.

# Our First Canvas

Let's create a `<canvas>` with `id="three-d-canvas"`.

We can do it either using raw markup below this very cell, or by using the `IPython.display` module which allows us to display rich representations of objects in a Jupyter notebook (much like the `Deno.jupyter.display` module we saw in the [D3.js US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb)).

Using `IPython.display` is more reliable.

In [13]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas'></canvas>"))

Here's the code that produces the above output. Feel free to expand and examine. We will paint the general strokes below.

<details><summary>Click to expand the Torus code</summary>
```javascript
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm'

const scene = new three.Scene();

const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

const canvas = document.getElementById("three-d-canvas")

const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
camera.position.setZ(30);

const renderer = new three.WebGLRenderer({
    canvas: canvas
});

renderer.setPixelRatio( window.devicePixelRatio );
renderer.setSize( bodyWidth, bodyHeight );


renderer.render(scene, camera);

const geometry = new three.TorusGeometry(10,3,16,100);
const material = new three.MeshBasicMaterial({ color: 0xFF6347, wireframe: true });
const torus = new three.Mesh(geometry, material);

scene.add(torus);

function animate() {
    requestAnimationFrame(animate);

    torus.rotation.x += 0.01;
    torus.rotation.y += 0.005;
    torus.rotation.z += 0.01;

    renderer.render(scene, camera);
}

animate();
```
</details>

You'll notice some general strokes.

First, we create a `three.Scene` object.

```js
const scene = new three.Scene();
```

Then, we define some constants for Three.js's [WebGLRenderer](https://threejs.org/docs/#api/en/renderers/WebGLRenderer) relative to the HTML document's body (grabbing the latter using the DOM API).

```js
const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;
```

Then, we grab the `<canvas>` element we created.

```js
const canvas = document.getElementById("three-d-canvas")
```
After that, we create a [PerspectiveCamera](https://threejs.org/docs/#api/en/cameras/PerspectiveCamera), supplying it the [field of view](https://en.wikipedia.org/wiki/Field_of_view) among other attributes, and setting its position.

```js
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
camera.position.setZ(30);
```

We then create the `WebGLRenderer` object, supplying it the `canvas` to render, as well as setting some of its parameters.

```js
const renderer = new three.WebGLRenderer({
    canvas: canvas
});
renderer.setPixelRatio( window.devicePixelRatio );
renderer.setSize( bodyWidth, bodyHeight );
```

Finally, we render the scene by invoking the renderer's `.render(scene, camera)`.

```js
renderer.render(scene, camera);
```

The `scene` object continues to be our window into the 3D world we just created. To it, we add `geometries` and their `materials` through a combination object called a [Mesh](https://threejs.org/docs/#api/en/objects/Mesh).

```js
const geometry = new three.TorusGeometry(10,3,16,100);
const material = new three.MeshBasicMaterial({ color: 0xFF6347, wireframe: true });
const torus = new three.Mesh(geometry, material);

scene.add(torus);
```

We can also add a little life to the scene by using a custom animation function.
```js
function animate() {
    requestAnimationFrame(animate);

    torus.rotation.x += 0.01;
    torus.rotation.y += 0.005;
    torus.rotation.z += 0.01;

    renderer.render(scene, camera);
}
```
We can invoke the animation within the outer lexical environment, for now.

```
animate();
```

## Grid View

To display multiple 3D `scenes` in a grid, we can simply use CSS-grid inside Jupyter.

First we lay out the markup inside the notes:

```html
<div class="three-d-grid-container">
    <div class="three-d-grid-item"><canvas id="three-d-canvas-1"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-2"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-3"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-4"></canvas></div>
</div>
```

Then, we include the stylesheet in the post's own subdirectory, and use something like Quarto's `include-after-body` (as we also load the external scripts used to produce these rich 3D outputs):

```yaml
include-after-body:
    <script type="module" src="./javascript/three-js-demo.js"></script>
```

It turns out that there's support for including CSS inside a Quarto post. It's done using the `format.html` flag in the front-matter as follows:

```yaml
format:
  html:
    css: ./css/three-js-demo.css
```

The file `three-js-demo.css` should contain these minimal styles: 

```css
#three-d-grid-container {
    display: grid;
    grid-template-columns: repeat(2, 1fr);
    grid-template-rows: repeat(2, 1fr);
    gap: 10px;
    width: 100%;
    height: 50%;
}
.three-d-grid-item {
    display: flex;
}
```

Let's add the grid markup.

Now, inside the external script, we can reference the various `canvas` elements by `id`.

In [14]:
from IPython.display import display, HTML
display(HTML(
    """
    <div id='three-d-grid-container'>
        <div class="three-d-grid-item" id="three-d-grid-item-1"><canvas id="three-d-canvas-1"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-2"><canvas id="three-d-canvas-2"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-3"><canvas id="three-d-canvas-3"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-4"><canvas id="three-d-canvas-4"></canvas></div>
    </div>
    """
))

# Loading Custom Objects with Materials

Quarto won't serve `.obj` files (at least by default), so we can just commit the object file to the remote repository and use the `raw` GitHub link (as a CDN).

We will also need the `OBJLoader`, a Three.js add-on. This can be grabbed from a CDN as well. We may also include it in the `include-after-body.text` front matter *before* loading the script file itself as:

```yaml
<script type="importmap">
    {
        "imports": {
            "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
            "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js"
        }
    }
</script>
```
And then, within the script, import as follows:

```js
import * as three from 'three';
import { OBJLoader } from 'OBJLoader';
```

Here's the full code to render a rubber ducky. It will help us debug our code going forward. I downloaded this model from [Sketchfab](https://sketchfab.com/3d-models/rubber-duck-ecb9ce9ff973406398ee56e391f9c902) which hosts many such free models.

<details><summary> Click to expand the code used to generate the rubber ducky</summary>

```js
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/build/three.module.js';
import { OBJLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js';
import { MTLLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js';

// Get the container and set dimensions
const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

// Set up the canvas, scene, camera, and renderer
const canvas = document.getElementById("three-d-canvas");
const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({ canvas: canvas });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);

// Position the camera so the object will be in view.
camera.position.set(0, 20, 0);

// Add some lights so the materials are visible.
const ambientLight = new three.AmbientLight(0xffffff, 0.6);
scene.add(ambientLight);

const directionalLight = new three.DirectionalLight(0xffffff, 0.8);
directionalLight.position.set(10, 20, 10);
scene.add(directionalLight);

// Load the MTL file first.
const mtlLoader = new MTLLoader();
mtlLoader.load(
  'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.mtl',
  (materials) => {
    materials.preload();

    // Now load the OBJ file and set its materials.
    const objLoader = new OBJLoader();
    objLoader.setMaterials(materials);
    objLoader.load(
      'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.obj',
      (object) => {
        // Scale and position the loaded object.
        object.scale.set(5, 5, 5);
        object.position.set(0, 15, -20);
        scene5.add(object);

        // Animation loop
        function animate() {
          requestAnimationFrame(animate);
          object.rotation.y += 0.01;
          renderer.render(scene, camera);
        }
        animate();
      },
      // onProgress callback
      (xhr) => {
        console.log("Loading object...");
      },
      // onError callback
      (error) => {
        console.error('An error occurred while loading the OBJ:', error);
      }
    );
  },
  // onProgress callback for MTL
  (xhr) => {
    console.log("Loading materials...");
  },
  // onError callback for MTL
  (error) => {
    console.error('An error occurred while loading the MTL:', error);
  }
);
```
</details>

Note that `OBJLoader` takes the URL of the object followed by a callback function that gets triggered by the object fully loading. 

The geometries of the object are in `object.children[n].geometry`. Each child is a part of the object (for example the eyes, wings, and nose of the duck correspond to the object's child geometries).

Let's create another canvas to render the rubber ducky.

In [19]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-5'></canvas>"))

For the materials, we used `MTLLoader`. Another add-on downloaded from the following CDN in the `include-after-body.text` front matter:

```yaml
<script type="importmap">
    {
        "imports": {
            "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js"
        }
    }
</script>
```
Notice the nested calls? `OBJLoader` is typically called within `MTLLoader`, and gets the `material` supplied to it.

The `MTLLoader` loads the material file (`rubber_duck.mtl`) and then calls `materials.preload()`. The `OBJLoader`’s `.setMaterials(materials)` ensures that when the OBJ file is loaded, it uses the material definitions from the MTL file.

# More Complex Scenes

HTML Canvas, by itself, is a thing of wonder. Here's a curated list of cool [Canvas API examples](https://github.com/raphamorim/awesome-canvas?tab=readme-ov-file). For example, here's [Pong](https://cssdeck.com/labs/full/ping-pong-game-tutorial-with-html5-canvas-and-sounds). Here's a cool [Matrix animation](https://matrix.dotglitch.dev/), and here's a tool that visualizes [L-systems](https://www.kevs3d.co.uk/dev/lsystems/#). The Canvas is what makes things like drawing tools or diagramming tools, such as Lucid, possible on the browser. 

For complex scenes, the HTML Canvas with the Canvas API is not enough. The [Canvas API](https://developer.mozilla.org/en-US/docs/Web/API/Canvas_API) is usually good for 2D applications (it explicitly sets its context as `"2d"`). This is where Three.js with its WebGL support comes in. 

For truly complex 3D web-applications (with mouse and keyboard controls and other advanced features or, perhaps, gameplay), we will need to integrate Three.js into a React app using [React Three Fiber (r3f)](https://r3f.docs.pmnd.rs/getting-started/introduction). React Three Fiber is a library that serves as a React renderer for Three.js. It enables the creation and management of 3D scenes and objects using React components, rather than plain HTML which allows developers to leverage React's ecosystem for state management, component reusability, effect management (through `useEffect`), and its DOM lifecycle (so as not to rely on the browser's events that signal certain phases of the document's life like `DOMContentLoaded`).

But for now, we will push the limits of Three.js in Jupyter using Quarto. Let's continue by adding many more objects. Let's add more rubber duckies to the scene. Then we will make it rain rubber duckies, which will require the use of physics! 

## Multiple Objects in a Scene

Previously we rendered separate scenes in a grid, now let's see how many ducks we can add to the same scene. 

Sure, we might think, let's just use `object.clone()` in the `OBJLoader` callback:

```js
// Clone the already loaded duck object
const duckClone = object.clone();
```

But here's where we first stumble onto major performance issues in the browser. Depending on how much RAM our machine has, we will start noticing jittery behavior and slowed animations when we try to render `50` or more of these rubber duckies. Luckily, Three.js has an optimization for rendering multiple of the same object to the scene (even if they have to be animated differently). 

### InstancedMesh

Instead of using `Mesh` we must used `InstancedMesh`. This optimization is called **instancing**. In Three.js this is implemented via the `InstancedMesh` class, as already mentioned, which lets us render many copies of the same geometry and material using a *single* draw call. Instead of creating a separate `Mesh` for each object, which incurs a draw call per object, we can create one `InstancedMesh` and assign each instance its own transformation matrix (and even per-instance colors, if needed). This technique reduces CPU-to-GPU communication overhead, making it ideal for rendering thousands of identical objects efficiently

We don’t call `clone()` for each duck when using instancing. Instead, we create one `InstancedMesh` that shares the *same* geometry and material for every instance and then assign each instance its own transformation matrix. A transformation matrix is just what it sounds like, it's a matrix supplied to `InstancedMesh` that applies a rotation, a translation, and a scaling transform. This sounds like it's too complicated to come up with, after all is this a Linear Algebra exercise? But it's very simple, and there are a lot of tricks to help us do just that! 

`InstancedMesh` is basically an indexed data structure that stores *instances* of the `InstancedMesh`. These instances aren't directly accessible, unlike clones. However, we get the next best thing: an interface to update the transformation matrices of the instances (or other instance attributes, like color). We have the getter `getMatrixAt(index, matrix)`, and the setter `setMatrixAt(index, matrix)`.

One question that may arise: Why does the getter require a `matrix` parameter? The design of the `getMatrixAt` method is such that we supply an existing `Matrix4` as a container. The method then writes the transformation matrix of the instance at index `i` into that provided matrix. 
Why use a $4 \times 4$ matrix in 3D space? Refer to the subsection below on [homogeneous-coordinates](#homogeneous-coordinates---linearaffine-transformations). We should avoid creating a new `Matrix4` object on every call and, instead, re-use one `dummy.matrix`.

Here's the code, feel free to expand and examine. We will also walk through the changes at the bottom.

<details><summary>Click to expand</summary>

```js
// NEW CODE WILL GO HERE
```

</details>

`InstancedMesh` works with a single `geometry` and `material`.
Since our OBJ is a group, we need to extract the mesh out of it in order to instance the mesh. 

The simplest way to do this is by taking the first mesh child `object.children[0]`.

```js
// Extract the mesh from OBJ
const duckMesh = object.children[0];
if (!duckMesh) {
  console.error('Loaded object does not contain a mesh.');
  return;
}
```
But this produces nothing other than the duck's wingless body without eyes or a beak. So we have to merge the geometries. This is done using the Three.js add-on [BufferGeometryUtils.mergeBufferGeometries](https://threejs.org/docs/#examples/en/utils/BufferGeometryUtils.mergeBufferGeometries). This requires an understanding of the OBJ representation.

#### The OBJ Representation of an Object

When Three.js gets an OBJ file, it represents it as this `Group`. 

<details><summary>Click to expand pseudo-json with comments</details>

```json
Group {
  "children": [
    "Mesh": {
      "geometry": "BufferGeometry" // See below for full expansion...
      "material": "Material" // See below for full expansion...
    },
    "Mesh": {
      "geometry": {
        "metadata": { // Metadata provides info about the export
          "version": 4.5, // Exporter version
          "type": "BufferGeometry", // Type of object
          "generator": "BufferGeometry.toJSON"// Method that generated this JSON
        },
        "uuid": "12345678-1234-1234-1234-123456789abc", // Unique identifier for this geometry
        "type": "BufferGeometry", // Confirms that this is a BufferGeometry
        "data": { // Main container for all geometry data
          "attributes": { // Vertex attributes (data arrays for each vertex property)
            "position": { // Positions of vertices
              "itemSize": 3, // Each vertex position has 3 components: x, y, z
              "type": "Float32Array", // Data stored as a Float32Array
              "array": [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0], // Example vertex positions
              "normalized": false // Data is not normalized
            },
            "normal": { // Normals at each vertex
              "itemSize": 3, // 3 components per normal (x, y, z)
              "type": "Float32Array",
              "array": [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1], // Example normals
              "normalized": false
            },
            "uv": { // Texture coordinates
              "itemSize": 2, // Each UV coordinate has 2 components: u, v
              "type": "Float32Array",
              "array": [0, 0, 1, 0, 1, 1, 0, 1], // Example UV values
              "normalized": false
            }
          },
          "index": { // Index data defines the order to connect vertices into triangles
            "type": "Uint16Array", // Data type of the index array
            "array": [0, 1, 2, 0, 2, 3] // Indices forming triangles
          },
          "groups": [ // Groups specify portions of the geometry that use different materials
            {
              "start": 0, // Starting index in the index array for this group
              "count": 6, // Number of indices (here, one quad made of 2 triangles)
              "materialIndex": 0  // Which material from the material array should be used
            }
          ]
        }
      },
      "material": {
        "metadata": { // Metadata about this material export
          "version": 4.5, // Version of the exporter
          "type": "Object", // This is a JSON object
          "generator": "Material.toJSON" // Generated by Material.toJSON method
        },
        "uuid": "a1b2c3d4-e5f6-7890-abcd-ef1234567890", // Unique ID for the material
        "type": "MeshStandardMaterial", // Type of material (could be MeshBasicMaterial, etc.)
        "color": 16777215, // Base color (in decimal, here 16777215 equals 0xffffff - white)
        "roughness": 0.5, // Roughness parameter (controls how rough the surface is)
        "metalness": 0.5, // Metalness parameter (how metallic the material looks)
        "emissive": 0, // Emissive color (self-illumination; 0 means black, no emission)
        "opacity": 1, // Opacity value (1 means fully opaque)
        "transparent": false, // Flag indicating whether the material supports transparency
        "wireframe": false, // If true, the material renders as a wireframe instead of solid
        "side": 2 // Which side of the faces to render (2 indicates DoubleSide)
      }
    }
  ],
  // Other group properties...
}
```

</summary>

In a `BufferGeometry`, the *index array*, which looks as below, is an optional array that defines how the vertices (stored in the attributes `"position"`) are connected to form faces (usually triangles). The indices (numbers) in an index array refer to vertices in the attributes.

```json
"index": {
  "type": "Uint16Array",
  "array": [0, 1, 2, 0, 2, 3]
}
```

For example, if our geometry's `"position"` attribute contains the following vertices:

```json
"position": {
  "itemSize": 3,
  "array": [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]
}
```

Then the index array above tells Three.js to use the vertices at positions `0, 1, 2` to form the first triangle, and vertices at positions `0, 2, 3` to form the second triangle.

The *starting index* of the group refers to the *offset* in the index array where a specific group (which might be assigned a particular material) begins. The `"group"` defines a subset of the index array that should be rendered with a specific material.

Now that we understand the structure of an object better, we can build a mesh merger that preserves materials.

#### Merging Meshes while Preserving Materials

We need to merge the child meshes while preserving each of their materials. Somehow, we have got to preserve information about which *parts* of the merged geometry should use which material. In Three.js this is done using `groups` inside a `BufferGeometry`. Each `group` defines a *start index*, a *count*, and a *material index*.
When using `mergeBufferGeometries`, we pass a second parameter as `true` to tell the function to preserve the groups from each individual geometry. Then we supply a materials array (in the same order that the groups refer to) when we create the Mesh.

Since merging meshes (while preserving the materials) is a thing we're going to do quite often, let's make a helper function called `mergeMeshes` that takes an array of `three.Mesh` objects, merges their geometries while preserving their material assignments via groups, and returns a single merged mesh. We'll store it as its own module and import it into our external scripts. Here's the implementation, see below it for an explanation of each step.

```js
import * as three from 'three';
import { mergeBufferGeometries } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/utils/BufferGeometryUtils.js';

/**
 * mergeMeshes accepts an array of three.Mesh objects and returns a single merged mesh.
 * It preserves each mesh's material by assigning groups to the merged geometry.
 *
 * @param {three.Mesh[]} meshes - Array of meshes to merge.
 * @returns {three.Mesh} - A new mesh that is the merged result.
 */
function mergeMeshes(meshes) {
  // Arrays to hold the individual geometries and their corresponding materials.
  const geometries = [];
  const materials = [];
  
  // materialIndex will be used to assign a unique material index for each mesh.
  let materialIndex = 0;
  
  // Loop through each mesh in the input array.
  meshes.forEach((mesh) => {
    // 1. Update the world matrix so that we capture the mesh's global transformation.
    mesh.updateWorldMatrix(true, false);

    // 2. Clone the mesh's geometry so we don't modify the original.
    const geom = mesh.geometry.clone();

    // 3. Apply the mesh's world transformation to the geometry. This "bakes" the mesh's position, rotation, and scale into its vertices.
    geom.applyMatrix4(mesh.matrixWorld);

    // 4. If the geometry doesn't already have groups defined (which tell us which part uses which material),
    //    we add a single group that covers the whole geometry.
    //    This group assigns a material index that corresponds to the mesh's material in our materials array.
    if (geom.groups.length === 0) {
      // Determine the number of elements (either from the index count or the vertex count)
      const count = geom.index ? geom.index.count : geom.attributes.position.count;
      // Add a group from start=0 to count with the current material index.
      geom.addGroup(0, count, materialIndex);
    }

    // 5. Push the processed geometry and the mesh's material into our arrays.
    geometries.push(geom);
    materials.push(mesh.material);
    materialIndex++;
  });

  // 6. Merge all the geometries into a single BufferGeometry.
  //    The second argument 'true' tells the function to preserve the groups from each geometry.
  const mergedGeometry = mergeBufferGeometries(geometries, true);

  // 7. Create a new Mesh using the merged geometry and the array of materials.
  //    THREE.Mesh accepts an array of materials when the geometry contains groups with material indices.
  const mergedMesh = new THREE.Mesh(mergedGeometry, materials);
  
  // Return the merged mesh.
  return mergedMesh;
}
```

First, inside the loop that iterates over each child mesh, we update the mesh's `matrixWorld`. Calling `mesh.updateWorldMatrix(true, false)` ensures that the mesh’s **global transformation** (its position, rotation, and scale) is up-to-date. In short, updating the world matrix is necessary so that the transformation we apply to the geometry truly represents the mesh’s position, rotation, and scale within the entire scene. Each mesh as its own **local transformation** (which is exclusively applied to it), but meshes can also be in a group with its own local transformation. The final transformation, the `matrixWorld`, is the application of all these local transformation matrices. Calling `mesh.updateWorldMatrix(true, false)` just ensures this calculation is up-to-date.

```js
mesh.updateWorldMatrix(true, false);
```
It's good practice to clone the mesh's geometry so that we don't alter the original.

```js
const geom = mesh.geometry.clone();
```

Note that earlier we cloned the entire object with [three.Object3D.clone](https://threejs.org/docs/#api/en/core/Object3D.clone), but now we're cloning just the geometry of the mesh using [three.BufferGeometry.clone](https://threejs.org/docs/#api/en/core/BufferGeometry.clone).

Next step is to apply the updated world matrix (transformation) by calling `geom.applyMatrix4(mesh.matrixWorld)`. This is known as *baking-in* the mesh's world transformation into the geometry. Don't get discouraged if you don't understand transformations, all that these transformations mean, essentially, is that the vertices of the geometry are moved to their correct positions in the world space.

```js
geom.applyMatrix4(mesh.matrixWorld);
```

We then assign groups (if absent). Groups in `BufferGeometry` indicate which sections of the geometry use which material. If a mesh’s geometry doesn’t already have groups, we add one covering the entire geometry and assign it the current *material index*. This is essential for preserving different materials when the geometries are merged.



1. **Collect Geometries and Materials:**  
   We store each processed geometry and its corresponding material in separate arrays. The order is important because the group’s material index in each geometry will correspond to the position of the material in the `materials` array.

2. **Merge Geometries:**  
   `mergeBufferGeometries(geometries, true)` merges all collected geometries into a single geometry, preserving the groups. This is what reduces the number of draw calls during rendering.

3. **Create Merged Mesh:**  
   Finally, we create a new THREE.Mesh with the merged geometry and the array of materials. The merged geometry’s groups ensure that the correct material is used for each part of the merged mesh.

You can now call this function with any array of meshes (for example, the children of a loaded OBJ) to obtain a single, optimized mesh with preserved materials.

----- I AM HERE -----


Once we have the mesh extracted out of the OBJ, we create an `InstancedMesh` with `numDuckies` number of ducks, and supplying the mesh `geometry` and `material`.

```js
const numDuckies = 200;

// Create an InstancedMesh using the duck's geometry and material.
const instancedDuck = new three.InstancedMesh(
  duckMesh.geometry,
  duckMesh.material,
  numDuckies
);
```
As we would do with regular objects (like clones), we need to add the `InstancedMesh` to the scene using:

```js
scene.add(instancedDuck);
```
The next step involves an industry trick. Instead of coming up with a transformation matrix ourselves, we create a `dummy` object and use its `.position.set(x,y,z)` as always. We give it some rotation (like before). Then we do `.updateMatrix()` and use its `.matrix` to *get* the object's transformation matrix. 

Once we have this `dummy` object's transformation matrix, we provide it to the `.setMatrixAt(index, matrix)` method of the `InstancedMesh`.

```js
// Create a dummy Object3D to build transformation matrices.
const dummy = new three.Object3D();

// Initialize each instance with a random position and rotation.
for (let i = 0; i < numDuckies; i++) {
  dummy.position.set(
    randInRange(-30, 30),
    randInRange(-30, 30),
    randInRange(-30, 30)
  );
  dummy.rotation.y = randInRange(0, Math.PI * 2);
  dummy.updateMatrix();
  instancedDuck.setMatrixAt(i, dummy.matrix);
}
```
Finally, we need to update the `animate()` function to apply a $y$-rotation to each individual instance. This is where `matrix.decompose(position, rotation, scale)` comes in. It does what it sounds like it does! It decomposes a $4 \times 4$ transformation matrix into its constituent transformations: capturing the translation, rotation, and scale.

```js
// Animate: update rotation for each instance.
function animate(time) {
  requestAnimationFrame(animate);

  // Rotate each duck around its own y-axis.
  for (let i = 0; i < numDuckies; i++) {
    // Retrieve the current matrix of instance i.
    instancedDuck.getMatrixAt(i, dummy.matrix);
    // Decompose the matrix into position, rotation, and scale.
    dummy.matrix.decompose(dummy.position, dummy.rotation, dummy.scale);
    // Increment the rotation.
    dummy.rotation.y += 0.01;
    // Update the dummy's matrix.
    dummy.updateMatrix();
    // Set the new matrix for instance i.
    instancedDuck.setMatrixAt(i, dummy.matrix);
  }
  // Mark the instance matrix attribute as needing an update.
  instancedDuck.instanceMatrix.needsUpdate = true;
  renderer6.render(scene6, camera6);
}
```

### Homogeneous Coordinates - Linear/Affine Transformations 

Why do 3D graphics use a $4 \times 4$ matrix (`Matrix4`) rather than a $3 \times 3$ matrix to represent transformations in 3D space? A $3 \times 3$ matrix can handle linear transformations like rotation and scaling, but it cannot handle affine transformations like a simple translation. It also can't handle projection (**perspective** or **orthogonal** -- corresponding to non-linear and linear transformations). 

The $4 \times 4$ matrix incorporates an extra row and column that stores the translation (or projection) components, enabling *all* simple transformations to be combined into one matrix. This makes it very efficient for computer graphics since we can apply a single matrix to transform an object in 3D space, reducing the number of required matrix operations. 

The use of a $4 \times 4$ matrix necessitates the use of **homogeneous coordinates** (which all 3D libraries do under the hood). This is when the normal $(x,y,x)$ coordinates are augmented as $(x,y,z,w)$ (where $w$ is an extra coordinate that's $1$ by default). Let's see how this helps.
Suppose we want to translate a point by $(t_x, t_y, t_z)$. In homogeneous coordinates, the translation matrix is:

$$
T = \begin{pmatrix}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z \\
0 & 0 & 0 & 1
\end{pmatrix}
$$

Now, take a point $P$ represented as:

$$
P = \begin{pmatrix} x \\ y \\ z \\ 1 \end{pmatrix}
$$

When we apply the translation matrix (identity transformation) to the point, we multiply:

$$
T P = \begin{pmatrix}
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z \\
0 & 0 & 0 & 1
\end{pmatrix}
\begin{pmatrix} x \\ y \\ z \\ 1 \end{pmatrix}
=
\begin{pmatrix}
x + t_x \\
y + t_y \\
z + t_z \\
1
\end{pmatrix}
$$

This shows that the point $(x, y, z)$ is translated to $(x+t_x, y+t_y, z+t_z)$. We simply mentally discard the extra coordinate.

In summary, the extra coordinate $w$ in homogeneous coordinates enables us to include translation (and projection) in the *same* framework as other transformations.

-----

```js
// An array to store our duck clones
const duckClones = [];

// The number of duckies we want
const numDuckies = 50;

// Function to generate a random number in a provided range
function randInRange(min, max) {
  return Math.random() * (max - min) + min;
}

for (let i = 0; i < numDuckies; i++) {
  // Clone the loaded duck object
  const duckClone = object.clone();

  // Randomize position (for example, within a certain X/Z range and different Y values)
  duckClone.position.set(
    randInRange(-50, 50),  // Random x position between (-50,50)
    randInRange(0, 20),    // Random y position 
    randInRange(-50, 50)   // Random z position
  );

  // Optionally, randomize rotation so they face different directions.
  duckClone.rotation.y = randInRange(0, Math.PI * 2);

  // Add the clone to the scene and our array.
  scene.add(duckClone);
  duckClones.push(duckClone);
};
```

This is a good time to make an effort to understand the coordinate system in Three.js. How do we know which specific range of positions to supply `randInRange`? We can actually display the axes and coordinate grid. Click the note below to see how.

::: {.callout-tip title="💡 TIP: Displaying `AxesHelper` and `GridHelper` in the Scene" appearance="minimal" collapse="true"}

The `AxesHelper` displays lines for the x, y, and z axes (typically colored red, green, and blue, respectively). For example:

```js
// An AxesHelper with a size of 50 units
const axesHelper = new three.AxesHelper(50);
scene.add(axesHelper);
```
This helper will render lines originating from the scene’s origin $(0, 0, 0)$, so we can visually see how objects are positioned relative to it.

If we want an additional visual cue that represents a grid on the ground (helpful for orienting objects in a scene), we can use the `GridHelper`:

```js
// A GridHelper with a grid size of 100 and 10 divisions
const gridHelper = new three.GridHelper(100, 10);
scene.add(gridHelper);
```

Finally, if our grid or axes are not showing up it may be due to the camera's position. We can point the camera to any vector!

```js
camera.lookAt(new three.Vector3(0, 0, 0));
```
:::

Let's create another canvas for our new scene!

In [21]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-6'></canvas>"))